The goal of this notebook is to make any api calls and save the results to a csv/json format so that we can read the responses without re-running the api calls and we don't need to track api keys. 

In [1]:
#imports including api_key if needed
#because no api key is needed for the world bank api, no need to change .gitignore or import from config.py

import pandas as pd
import numpy as np
import requests
import json
import time
from pathlib import Path

### Sources for API Calls for Population

uses of open data - API to get the population by country - Open Data Stack Exchange
https://opendata.stackexchange.com/questions/20863/api-to-get-the-population-by-country

Developer Information – World Bank Data Help Desk
https://datahelpdesk.worldbank.org/knowledgebase/topics/125589

Country API Queries – World Bank Data Help Desk
https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries


From above links:


Requesting Country Data
To list all countries: http://api.worldbank.org/v2/country

The following information will appear, when available, in the response when using this country query through the World Bank API:

3 letter ISO 3166-1 alpha-3 code
2 letter ISO 3166-1 alpha-2 code
Name
Region: ID, name and World Bank 2 letter code
Income Level: ID, name and World Bank 2 letter code
Lending Type: ID, name and World Bank 2 letter code
Capital City
Longitude
Latitude



Sample Request Format: Country Query
For XML format: http://api.worldbank.org/v2/country/br

For JSON format: http://api.worldbank.org/v2/country/br?format=json

Note: “br” is the two-letter ISO code for Brazil.

Sample response for JSON format country query for Brazil:

```
[
  {
    "page": 1,
    "pages": 1,
    "per_page": "50",
    "total": 1
    },
    [
      {
        "id": "BRA",
        "iso2Code": "BR",
        "name": "Brazil",
        "region": {
          "id": "LCN",
          "iso2code": "ZJ",
          "value": "Latin America & Caribbean (all income levels)"
        },
        "adminregion": {
          "id": "LAC",
          "iso2code": "XJ",
          "value": "Latin America & Caribbean (developing only)"
        },
        "incomeLevel": {
          "id": "UMC",
          "iso2code": "XT",
          "value": "Upper middle income"
        },
        "lendingType": {
          "id": "IBD",
          "iso2code": "XF",
          "value": "IBRD"
        },
        "capitalCity": "Brasilia",
        "longitude": "-47.9292",
        "latitude": "-15.7801"
      }
    ]
  ]
  ```


API Basic Call Structures – World Bank Data Help Desk
https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures

Gives us the following query to get population:

https://api.worldbank.org/v2/country/br/indicator/SP.POP.TOTL?format=json

which gives us output like:

```

[{"page":1,"pages":2,"per_page":50,"total":63,"sourceid":"2","lastupdated":"2023-12-18"},[{"indicator":{"id":"SP.POP.TOTL","value":"Population, total"},"country":{"id":"BR","value":"Brazil"},"countryiso3code":"BRA","date":"2022","value":215313498,"unit":"","obs_status":"","decimal":0},{"indicator":{"id":"SP.POP.TOTL","value":"Population, total"},"country":{"id":"BR","value":"Brazil"},"countryiso3code":"BRA","date":"2021","value":214326223,"unit":"","obs_status":"","decimal":0},{"indicator":{"id":"SP.POP.TOTL","value":"Population, total"},"country":{"id":"BR","value":"Brazil"},"countryiso3code":"BRA","date":"2020","value":213196304,"unit":"","obs_status":"","decimal":0},{"indicator":{"id":"SP.POP.TOTL","value":"Population, total"},"country":{"id":"BR","value":"Brazil"},"countryiso3code":"BRA","date":"2019","value":211782878,"unit":"","obs_status":"","decimal":0},
...
]]

```


This gives us country population data by year, although it is very messy to grab it. Might take some trial and error.

Looks like it returns a list. The second element of that list is a list. The elements of that list are dictionaries corresponding to countries, one for each country per year.
Keys for each dictionary in that list are indicator, country, countryiso3code, date, value, unit, obs_status, decimal. 
We only care about date and value, as the query will be generated for a specific country code. 

It also looks like there might be some page navigation, as that information is included in the first list entry of the return. It shouldn't be needed, we only need to look at the country entries for date=2024, 2023, 2022, 2021, 2020 (although 2024 and 2023 data are not yet available).

Unfortunately, our dataset is primarily data regarding 2022-2024, with the majority at this time being data from 2023. For this reason, I am choosing to find indicators for 2022, as that is the most recent indicator available. 



But I should be able to generate some visuals for country data science job listings from ai-jobs.net, which is the source of the data set we are using. 

Limitations in posts to that job listing site might limit the effectiveness of these metrics. Poor saturation of job postings on that site in one country compared to another could create an unreliable basis for country specifc metrics. 

Some potential questions to answer using this and the other data set:

different salaries in different countries
(choose several indicators from world bank api and compare across the counties/data science salaries)
job postings in different countries
job postings per capita in different countries


In [2]:
#Pseudocode:

#grab all countries from our dataset (create a list of the unique entries in 'company_location')
#API call to list all countires and store the request as json
#create a list of two letter ISO country codes that correspond to the list of countries in our dataset
#create a list/dictionary to store country data

#for loop - for each country_code in that list, we want to make a specific country query
    #url = f"http://api.worldbank.org/v2/country/{country_code}?format=json"
    #country_response = requests.get(url).json()
    #store data in country_data

#create pandas dataframe from country_data list/dictionary we generated (see module 6)
#save the dataframe to a csv, save it as "worldbank_api_results.csv" in "resources" directory


#read the csv in the other notebook, can create dataframes and try merging if needed. Otherwise, maybe just find key statistics. 

In [2]:
#grab all countries from our dataset:

data_science_salaries_df = pd.read_csv(Path("../resources/data_science_salaries.csv"))
# data_science_salaries_df.head()

unique_countries_company_location = list(data_science_salaries_df['company_location'].unique())
unique_countries_employee_residence = list(data_science_salaries_df['employee_residence'].unique())


print(len(unique_countries_company_location))

unique_countries = unique_countries_company_location.copy()


for country in unique_countries_employee_residence:
    if country not in unique_countries_company_location:
        unique_countries.append(country)

# display(unique_countries_company_location)

# display(unique_countries_employee_residence)
        
print(str(len(unique_countries_company_location)) + " " + str(len(unique_countries)) + " " + str(len(unique_countries_employee_residence)))

#so we have 89 countries to check to exhaust all references in coompany location/employee residence


75
75 89 87


In [3]:
#api calls for all countries in list:
#gather all country data so we can get the two letter codes for each country in our country list:

#took some time to figure out how to include all results into one query

all_countries_url = "https://api.worldbank.org/v2/country/all?format=json&per_page=300"
api_all_country_data = requests.get(all_countries_url).json()
# print(json.dumps(api_all_country_data, indent=4))

In [5]:
#we want the 2 letter iso code for that country, we can create a df with data for each country

#using company location as our location

#despite unique_countries having 89 values, country_ids comes up with a shorter list

country_ids = {}

for country in unique_countries:
    for country_api_call_data in api_all_country_data[1]:
        if country_api_call_data['name'] == country:
            country_ids[country] = country_api_call_data['iso2Code']

display(country_ids)
    

{'United States': 'US',
 'Romania': 'RO',
 'Portugal': 'PT',
 'Lebanon': 'LB',
 'India': 'IN',
 'Ireland': 'IE',
 'United Kingdom': 'GB',
 'Spain': 'ES',
 'Germany': 'DE',
 'Canada': 'CA',
 'Australia': 'AU',
 'Ukraine': 'UA',
 'Thailand': 'TH',
 'South Africa': 'ZA',
 'Slovenia': 'SI',
 'Singapore': 'SG',
 'Sweden': 'SE',
 'Saudi Arabia': 'SA',
 'Qatar': 'QA',
 'Poland': 'PL',
 'Philippines': 'PH',
 'New Zealand': 'NZ',
 'Norway': 'NO',
 'Netherlands': 'NL',
 'Nigeria': 'NG',
 'Mexico': 'MX',
 'Latvia': 'LV',
 'Luxembourg': 'LU',
 'Lithuania': 'LT',
 'Kenya': 'KE',
 'Japan': 'JP',
 'Italy': 'IT',
 'Israel': 'IL',
 'Hungary': 'HU',
 'Croatia': 'HR',
 'Greece': 'GR',
 'Gibraltar': 'GI',
 'Ghana': 'GH',
 'France': 'FR',
 'Finland': 'FI',
 'Estonia': 'EE',
 'Ecuador': 'EC',
 'Denmark': 'DK',
 'Colombia': 'CO',
 'Switzerland': 'CH',
 'Central African Republic': 'CF',
 'Brazil': 'BR',
 'Bosnia and Herzegovina': 'BA',
 'Armenia': 'AM',
 'Argentina': 'AR',
 'Andorra': 'AD',
 'United Arab Emir

In [6]:
missing_countries = []

for country in unique_countries:
    if country not in country_ids:
        missing_countries.append(country)

print(missing_countries)

#these countries still technically exist in our original dataset, we just have to find their iso codes and manually add them to the country_ids dictionary
#https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2

['Egypt', 'Vietnam', 'Turkey', 'Russia', 'South Korea', 'Hong Kong', 'Czech Republic', 'Iran', 'Bahamas', 'Jersey']


In [7]:
#Handling special cases where country names don't align

#egypt (arab republic of): eg
#vietnam (viet nam): vn
#turkey (turkiye) with special chars: tr
#russia (russian federation): ru
#south korea (republic of korea): kr
#hong kong (hong kong SAR): hk
#cezech republic, now czechia: cz
#iran (islamic republic of): ir
#bahamas: bs

#unfortunately, while listed in the world bank database, it is incomplete and no endpoint is setup for jersey
#jersey: je

#so we will not add jersey, but everything else can be added
missing_countries.pop()
display(missing_countries)

missing_country_ids = ['eg', 'vn', 'tr', 'ru', 'kr', 'hk', 'cz', 'ir' ,'bs']

for country, id in zip(missing_countries, missing_country_ids):
    country_ids[country] = id


#DOES NOT INCLIDE JERSEY
country_ids

['Egypt',
 'Vietnam',
 'Turkey',
 'Russia',
 'South Korea',
 'Hong Kong',
 'Czech Republic',
 'Iran',
 'Bahamas']

{'United States': 'US',
 'Romania': 'RO',
 'Portugal': 'PT',
 'Lebanon': 'LB',
 'India': 'IN',
 'Ireland': 'IE',
 'United Kingdom': 'GB',
 'Spain': 'ES',
 'Germany': 'DE',
 'Canada': 'CA',
 'Australia': 'AU',
 'Ukraine': 'UA',
 'Thailand': 'TH',
 'South Africa': 'ZA',
 'Slovenia': 'SI',
 'Singapore': 'SG',
 'Sweden': 'SE',
 'Saudi Arabia': 'SA',
 'Qatar': 'QA',
 'Poland': 'PL',
 'Philippines': 'PH',
 'New Zealand': 'NZ',
 'Norway': 'NO',
 'Netherlands': 'NL',
 'Nigeria': 'NG',
 'Mexico': 'MX',
 'Latvia': 'LV',
 'Luxembourg': 'LU',
 'Lithuania': 'LT',
 'Kenya': 'KE',
 'Japan': 'JP',
 'Italy': 'IT',
 'Israel': 'IL',
 'Hungary': 'HU',
 'Croatia': 'HR',
 'Greece': 'GR',
 'Gibraltar': 'GI',
 'Ghana': 'GH',
 'France': 'FR',
 'Finland': 'FI',
 'Estonia': 'EE',
 'Ecuador': 'EC',
 'Denmark': 'DK',
 'Colombia': 'CO',
 'Switzerland': 'CH',
 'Central African Republic': 'CF',
 'Brazil': 'BR',
 'Bosnia and Herzegovina': 'BA',
 'Armenia': 'AM',
 'Argentina': 'AR',
 'Andorra': 'AD',
 'United Arab Emir

In [8]:
#indicators of interest
#all options seen here: https://api.worldbank.org/v2/sources
#can find indicator values in metadata after selecting series here: https://databank.worldbank.org/source/world-development-indicators

#could find about a billion indicators if we wanted to
#computer/tech services as a breakdown of service exports for example, might be telling

#another way to check if they have data for a specific country
#https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?view=chart




#GDP per capita:
#for a given country code (us here) and date (2022 here)
# https://api.worldbank.org/v2/country/us/indicator/NY.GDP.PCAP.CD?date=2022&format=json



#income level classification can be found from the country query with no indicators:
# https://api.worldbank.org/v2/country/us?format=json


#population
#for a given year for country code = us, population api request:
#https://api.worldbank.org/v2/country/us/indicator/SP.POP.TOTL?date=2022&format=json


#we can try to show a derived statistic - jobs in our database for a specific year compared to country population total - would like to see if it's an accurate metric for the breakdown of the economy. More/less technical. Could test correlation with the following indicator? 


# here's a neat indicator:
# High technology exports as a % of manufactured goods
# https://data.worldbank.org/indicator/TX.VAL.TECH.MF.ZS?view=chart


#indicators don't always have data for every country! might have holes


In [9]:
#test api responses:


# base_url = "https://api.worldbank.org/v2/country/"
# gdp_per_capita_url = "/indicator/NY.GDP.PCAP.CD"
# population_url = "/indicator/SP.POP.TOTL"
# tech_manufacturing_url = "/indicator/TX.VAL.TECH.MF.ZS"
# end_url = f"?date={'2022'}&format=json"

# carlos = requests.get(url=f"{base_url}{'us'}{gdp_per_capita_url}{end_url}").json()
# print(carlos[1][0]['value'])

In [10]:
#let's start to build our dataframe from the api calls:

#we can build it by generating dictionaries with the key = column name, and calling the constructor
#only finding data for 2022, as other data doesn't seem to exist
date_for_metrics = '2022'

country_names = []
country_dates = []
#measured in current USD at time of data creation, ~2023
country_gdp_per_capitas = []
country_iso2codes = []
country_populations = []
country_tech_manufacturing_percentages = []
#country development index?

base_url = "https://api.worldbank.org/v2/country/"
gdp_per_capita_url = "/indicator/NY.GDP.PCAP.CD"
population_url = "/indicator/SP.POP.TOTL"
tech_manufacturing_url = "/indicator/TX.VAL.TECH.MF.ZS"
#country development index?


end_url = f"?date={date_for_metrics}&format=json"

for country, id in country_ids.items():
    country_names.append(country)
    country_dates.append(date_for_metrics)
    country_iso2codes.append(id)
    try:
        gdp_per_capita_response = requests.get(url=f"{base_url}{id}{gdp_per_capita_url}{end_url}").json()
        country_gdp_per_capitas.append(gdp_per_capita_response[1][0]['value'])
    
        population_response = requests.get(url=f"{base_url}{id}{population_url}{end_url}").json()
        country_populations.append(population_response[1][0]['value'])

        tech_manufacturing_response = requests.get(url=f"{base_url}{id}{tech_manufacturing_url}{end_url}").json()
        country_tech_manufacturing_percentages.append(tech_manufacturing_response[1][0]['value'])

        #country development index?
    except:
        print(f"Error in data collection for {country}. Data won't be aligned, so adding this country to a list of flagged countries, to be removed later. ")
        
        print(str(len(country_names)) + " " + str(len(country_dates)) + " " + str(len(country_iso2codes)) + " " + str(len(country_gdp_per_capitas)) + " " + str(len(country_populations)) + " " + str(len(country_tech_manufacturing_percentages)))




In [11]:
#test results

#conclusion: perfect, can make df without worrying much. Data should be aligned, but might have some 'Nones' that we will have to clean up depending on our analysis

print(country_tech_manufacturing_percentages)
print(country_names)



[17.850453747266, 11.6139339634163, 6.15477404783862, 15.1331223713952, 12.481728290332, 40.6897426792785, 26.5231369884372, 12.4147558313223, 15.9912089015793, 13.6696656141553, 25.6077480937055, 5.73612895811364, 21.7877759951296, 5.47917463897546, 8.4745670829046, 25.0082213617632, 16.8303208751309, None, 2.85120879871051, 10.7656426156211, 66.6110410778548, 11.9613716255842, 23.6088461741665, 20.5543341528917, 2.37962253520667, 19.3515236192737, 16.3914094262445, 5.86625622899729, 12.8208600979954, 2.31687195995541, 13.3699555327672, 8.76975456583408, 21.8337180558964, 17.7171070867763, 11.9755832912328, 14.260086601971, None, None, 20.7563596471658, 8.13242103758054, 17.6792610415742, 6.71157923405729, 15.8053906702031, 8.37821815596876, 28.8408410835182, 37.2974046773952, 9.04578247691606, 5.1638033162625, 21.407760574311, 4.7515752362265, 11.6593862348033, 9.28560107183941, None, 1.43131733752554, 28.2093260920791, 7.94130338987218, None, 21.8780511461657, 16.4974506531675, 7.92

In [12]:
list_columns = [country_names, country_iso2codes, country_dates, country_populations, country_gdp_per_capitas, country_tech_manufacturing_percentages] 

# world_bank_data_df = pd.DataFrame.from_dict({'country_name': country_names, 'country_id': country_ids, 'year' : country_dates, 'population': country_populations, 'gdp_pc': country_gdp_per_capitas, 'tech_manu_percent': country_tech_manufacturing_percentages})

world_bank_data = pd.Series(col for col in list_columns)

row_list = []
for row in world_bank_data:
    row_list.append(row)


world_bank_data_df = pd.DataFrame(row_list).T
world_bank_data_df.columns = ['country', 'id', 'year', 'population', 'gdp_pc', 'tech_manu_percent']

# world_bank_data_df = pd.DataFrame([world_bank_data[0],world_bank_data[1],world_bank_data[2],world_bank_data[3],world_bank_data[4],world_bank_data[5]]).T

world_bank_data_df.head()


,country,id,year,population,gdp_pc,tech_manu_percent
0,United States,US,2022,333287557,76329.582265,17.850454
1,Romania,RO,2022,19047009,15786.801742,11.613934
2,Portugal,PT,2022,10409704,24515.265851,6.154774
3,Lebanon,LB,2022,5489739,None,15.133122
4,India,IN,2022,1417173173,2410.888021,12.481728


In [13]:
world_bank_data_df.to_csv(Path('../resources/world_bank_api_data.csv'))